In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
print("Loading Glove Model")
wordsList = []
wordVectors = []
with open('glove_6B_50d.txt','r',encoding = 'utf8') as ft:
    for line in ft:
        splitLine = line.split()
        wordsList.append(splitLine[0])
        wordVectors.append(np.array([float(val) for val in splitLine[1:]]))
wordVectors = np.array(wordVectors)
print("Done.",len(wordVectors)," words loaded!")

Loading Glove Model
Done. 400000  words loaded!


In [6]:
maxSeqLength = 10
numDimensions = 300

firstSentence = np.zeros((maxSeqLength), dtype='int32')

firstSentence[0] = wordList.index("i")
firstSentence[1] = wordList.index("thought")
firstSentence[2] = wordList.index("the")
firstSentence[3] = wordList.index("movie")
firstSentence[4] = wordList.index("was")
firstSentence[5] = wordList.index("incredible")
firstSentence[6] = wordList.index("and")
firstSentence[7] = wordList.index("inspiring")
print(firstSentence.shape)
print(firstSentence) 

(10,)
[   41   804     0  1005    15  7446     5 13767     0     0]


In [7]:
with tf.Session() as sess:
    print(tf.nn.embedding_lookup(wordVector, firstSentence).eval().shape)

(10, 50)
